# Dialogue Summarization EDA

이 노트북은 대회용 `train/dev/test` 대화 요약 데이터를 탐색하고,
모델링/전처리/프롬프트 설계를 위한 인사이트를 얻기 위한 EDA 계획 및 실행을 담습니다.

## 1. EDA 전체 계획

1. **데이터 구조 파악**  
   - `train/dev/test` 행 개수, 컬럼(`fname, dialogue, summary, topic`) 구조 확인  
   - topic 분포 (train/dev) 비교, test에 topic 유무 확인

2. **대화/요약 길이 통계**  
   - 문자/토큰 기준 길이 분포 (평균, 중앙값, 상/하위 quantile)  
   - encoder_max_len=1024, decoder_max_len=80 기준으로 잘리는 비율 추정  
   - 발화(turn) 개수 분포 (#Person1/#Person2 등장 회수)

3. **토픽/스타일 분석**  
   - topic별 평균 대화 길이 / 요약 길이 비교  
   - 대표적인 대화-요약 페어 몇 개 샘플링해서, 요약 스타일(존댓말/반말, 요약의 추상화/추출 비율) 파악  
   - 요약에서 자주 등장하는 키워드/표현 조사 (간단한 빈도/WordCloud 수준)

4. **train/dev 데이터 분포 차이 체크**  
   - 길이 분포, topic 분포, 자주 등장하는 단어 등 비교  
   - 분포가 비슷한지, dev가 특정 topic에 편향되어 있는지 확인

5. **모델/프롬프트 설계에 직접 연결되는 분석**  
   - 긴 대화에서 앞/뒤 어느 부분이 요약에 더 많이 반영되는지 (간단한 n-gram overlap 위치 분석)  
   - 스타일 프롬프트 후보 문구를 몇 개 정의하고, 요약 스타일과 매칭되는지 눈으로 검증  
   - KoBART/T5 토크나이저 기준 토큰 길이 분포 (추후 필요 시)

6. **제출 파일/파이프라인 sanity check**  
   - `prediction/*.csv`에서 `fname` 정렬/중복 여부, summary 공백/길이 확인  
   - 극단적으로 짧거나 긴 요약 몇 개 샘플링하여 품질/스타일 확인

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("data")
TRAIN_PATH = DATA_DIR / "train.csv"
DEV_PATH = DATA_DIR / "dev.csv"
TEST_PATH = DATA_DIR / "test.csv"

train_df = pd.read_csv(TRAIN_PATH)
dev_df = pd.read_csv(DEV_PATH)
test_df = pd.read_csv(TEST_PATH)

train_df.head(), dev_df.head(), test_df.head()

## 2. 데이터 구조 및 기본 통계

- 각 split의 행 개수, 컬럼, 결측치, topic 유무 등 기본 정보 확인

In [ ]:
print("Train shape:", train_df.shape)
print("Dev shape:", dev_df.shape)
print("Test shape:", test_df.shape)

print("\nTrain columns:", train_df.columns.tolist())
print("Dev columns:", dev_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())

print("\nTrain topic value_counts:\n", train_df.get("topic", pd.Series()).value_counts().head())
print("\nDev topic value_counts:\n", dev_df.get("topic", pd.Series()).value_counts().head())

## 3. 길이 분포 분석 (문자 기준)

- dialogue 길이, summary 길이의 기본 통계를 보고, encoder/decoder max length 설정이 적절한지 감각을 잡습니다.

In [ ]:
for split_name, df in [("train", train_df), ("dev", dev_df)]:
    df["dialogue_len_char"] = df["dialogue"].astype(str).str.len()
    df["summary_len_char"] = df.get("summary", "").astype(str).str.len()

    print(f"==== {split_name.upper()} ====")
    print("dialogue_len_char describe:\n", df["dialogue_len_char"].describe())
    print("summary_len_char describe:\n", df["summary_len_char"].describe())
    print()

## 4. 발화(turn) 수 분포

- `#Person1#`, `#Person2#` 패턴을 이용해 대화 turn 수 분포를 대략적으로 확인합니다.

In [ ]:
import re

def count_turns(text: str) -> int:
    return len(re.findall(r"#Person[0-9]+#", str(text)))

train_df["num_turns"] = train_df["dialogue"].apply(count_turns)
dev_df["num_turns"] = dev_df["dialogue"].apply(count_turns)

print("Train num_turns describe:\n", train_df["num_turns"].describe())
print("Dev num_turns describe:\n", dev_df["num_turns"].describe())

## 5. topic별 길이/스타일 차이 (추후 확장)

- topic이 존재한다면, topic별로 길이/turn 수/요약 길이 차이를 비교합니다.
- 필요 시 시각화(히스토그램, 박스플롯)를 추가할 수 있습니다.

In [ ]:
if "topic" in train_df.columns:
    topic_stats = train_df.groupby("topic")["dialogue_len_char", "summary_len_char", "num_turns"].describe()
    topic_stats.head()

## 6. 샘플 대화-요약 페어 확인

- 무작위로 몇 개를 뽑아서, 요약 스타일과 프롬프트 설계 방향을 눈으로 확인합니다.

In [ ]:
SAMPLE_N = 3
sample_rows = train_df.sample(SAMPLE_N, random_state=42)
for i, row in sample_rows.iterrows():
    print("==== SAMPLE ====\n")
    print("fname:", row.get("fname"))
    print("topic:", row.get("topic"))
    print("[DIALOGUE]\n", row["dialogue"][:1000])
    print("\n[SUMMARY]\n", row["summary"])
    print("\n\n")